In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import utils

In [2]:
events = utils.get_events()

In [3]:
events.head(1)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,month,day,hora
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0


In [4]:
usuarios_convirtieron = events.loc[events.event == 'conversion'].person.unique()

In [5]:
eventos_de_compradores = events.loc[events.person.isin(usuarios_convirtieron)]
eventos_de_no_compradores = events.loc[~events.person.isin(usuarios_convirtieron)]

In [6]:
#rankeo a los browser de los compradores y los no compradores por separado
browser_ranking_compradores = eventos_de_compradores.browser_version.value_counts().rank()
browser_ranking_no_compradores = eventos_de_no_compradores.browser_version.value_counts().rank()

In [7]:
#asigno a cada user un browser
con_browser = events.loc[events.browser_version.notnull()]
feature_browser = con_browser.groupby(['person']).browser_version.value_counts().unstack(fill_value=0)
feature_browser['browser'] = feature_browser.idxmax(axis=1)
features = pd.DataFrame(data=feature_browser.browser, columns=['browser'])
features.head(1)

,browser
person,
0008ed71,Chrome 66.0


In [8]:
#armo el dataframe con los features
features['ranking_browser_conversion'] = features.browser.map(browser_ranking_compradores)
features['ranking_browser_no_conversion'] = features.browser.map(browser_ranking_no_compradores)
features.browser = features.browser.astype('category').cat.codes
features.head()

,browser,ranking_browser_conversion,ranking_browser_no_conversion
person,,,
0008ed71,75,366.0,366.0
00091926,75,366.0,366.0
00091a7a,97,352.5,351.0
000ba417,75,366.0,366.0
000c79fe,120,365.0,365.0


In [9]:
features.to_csv("features_browser.csv")